### 영화리뷰 분석(NLP)
* 네이버 영화 평점을 분석
* 영화평과 긍정 부정적 표현에 대한 프로그래밍 접근
* 평가 말뭉치를 분석하여 인간의 언어에서 긍정 부정의 언어체계를 조사
* 이를 바탕으로 문장의 긍부정 표현을 도출함

#### 자연 언어 처리(Natural Language Processing, 이하 NLP)
* 컴퓨터와 인간 언어 사이의 상호 작용하는 기술
* 인공지능의 핵심 기능 중 하나 
* 1950년대부터 기계 번역과 같은 자연어 처리 기술이 연구되기 시작
* 1990년대 이후에는 대량의 말뭉치(corpus) 데이터를 활용
* 기계학습 기반 및 통계적 자연어 처리 기법이 주류
* 최근에는 딥러닝과 딥러닝기반의 자연어처리가 방대한 텍스트로부터 의미 있는 정보를 추출하고 활용하기 위한 언어처리 연구 개발이 전 세계적으로 활발히 진행
* NLP 기술은 기계번역, 대화체 질의응답 시스템 대화시스템
* 정보검색, 말뭉치 구축, 시맨틱웹, 텍, 딥러닝, 
* 빅데이터 분석 분야뿐만 아니라 인간의 언어정보처리 원리와 이해를 위한 언어학과 뇌인지 언어정보처리 분야까지 핵심적인 요소로 작용

### 데이터 수집
- URL = https://movie.naver.com/movie/point/af/list.nhn?&page=1

In [170]:
# <!DOCTYPE html> 로 시작하지 않는 페이지는 BeautifulSoup 사용 불가. Selenium 사용해야 됨
import requests
from bs4 import BeautifulSoup
import cx_Oracle

In [171]:
# oracle DB 접속
uid='PROJECT1'
upw='Test1234'
url='localhost:1521/orcl'
conn=cx_Oracle.connect(uid,upw,url)
cursor=conn.cursor()

In [172]:
#data=[20200812, 19250, 20100, 19050, 19550, 8846552]
def insertReview(data=[]):
    sql="insert into MOVIEREV (MIDX,MTITLE,MCONTENT,MPOINT,MDATE,MUSER)  values({0},'{1}','{2}',{3},{4},{5})"
    sql1=sql.format(*data)
    try:
        cursor.execute(sql1)
        return(1)
    except:
        print(cursor.getbatcherrors())
        for error in cursor.getbatcherrors():
             print("Error", error.message.rstrip())
        print("[error]",sql1)
        return(0)
#insertStock(data)

In [173]:
def addData(data):
    sql="select * from MOVIEREV where MIDX="+str(data[0])
    cursor.execute(sql)
    res=cursor.fetchall()
    if(len(res)>0):
        print("데이터 중복")
    else:
        #print("입력가능 ")
        insertReview(data)
        conn.commit()
        print(data[0]," 입력완료 ")
#addData(data)

In [174]:
def setData(rev):
    midx=rev.select('.ac')[0].get_text()
    mtitle=rev.select('td.title a.movie')[0].get_text()
    mpoint=rev.select('td.title div.list_netizen_score em')[0].get_text()

    contSrc = str(rev.select('td.title')[0])
    brPos = contSrc.find('<br/>')
    frCont = contSrc[brPos+5:]
    endPos = frCont.find('<a class="report"')

    mcontent=frCont[:endPos].strip()

    td2=rev.select('td')[2]
    dateSrc=td2.get_text()

    mdate='20'+dateSrc[-8:].replace('.','')

    userSrc= str(td2)
    prePos = userSrc.find("list('nickname','")
    preSrc = userSrc[prePos+17:]
    endPos = preSrc.find("', 'after');")
    muser = preSrc[:endPos]
    
    mtitle=mtitle.replace("'"," ").replace('"',' ')
    mcontent=mcontent.replace("'"," ").replace('"',' ')
    
    t=[midx,mtitle,mcontent,mpoint,mdate,muser]
    if(mcontent!=''):
        addData(t)
    print('*' * 30)

In [175]:
def getReview(url):   
    req=requests.get(url)
    res=req.text[:200]
    if(len(res)>50):
        html=req.text
        soup=BeautifulSoup(html,'html.parser')
        src=soup.select('table.list_netizen  tbody tr')
        ## print(len(src)) ## 유일한 요소인가? 
        print(len(src))
        for rev in src:
            setData(rev)

In [176]:
sUrl="https://movie.naver.com/movie/point/af/list.nhn?&page="
for i in range(1,100):
    url = sUrl+str(i)
    getReview(url)

10
17071227  입력완료 
******************************
17071226  입력완료 
******************************
17071225  입력완료 
******************************
******************************
17071223  입력완료 
******************************
******************************
17071221  입력완료 
******************************
17071220  입력완료 
******************************
17071219  입력완료 
******************************
17071218  입력완료 
******************************
10
17071217  입력완료 
******************************
17071216  입력완료 
******************************
17071215  입력완료 
******************************
17071214  입력완료 
******************************
17071213  입력완료 
******************************
17071212  입력완료 
******************************
17071211  입력완료 
******************************
17071209  입력완료 
******************************
17071208  입력완료 
******************************
17071207  입력완료 
******************************
10
17071206  입력완료 
******************************
17071204  입력완료 
********************